# TfidfVectorizer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = ['Hello Soumyo Pandey here, I love machine learning','Welcome to the Fake News Detection Hub' ]

In [4]:
vect = TfidfVectorizer()

In [5]:
vect.fit(text)

TfidfVectorizer()

In [6]:
print(vect.idf_)

[1.40546511 1.40546511 1.40546511 1.40546511 1.40546511 1.40546511
 1.40546511 1.40546511 1.40546511 1.40546511 1.40546511 1.40546511
 1.40546511 1.40546511]


In [7]:
print(vect.vocabulary_)

{'hello': 2, 'soumyo': 10, 'pandey': 9, 'here': 3, 'love': 6, 'machine': 7, 'learning': 5, 'welcome': 13, 'to': 12, 'the': 11, 'fake': 1, 'news': 8, 'detection': 0, 'hub': 4}


# A words which is present in all the data, it will have low IDF value. With this unique words will be highlighted using the Max IDF values.

In [8]:
ex = text[0]
ex

'Hello Soumyo Pandey here, I love machine learning'

In [9]:
ex = vect.transform([ex])
print(ex.toarray())

[[0.         0.         0.37796447 0.37796447 0.         0.37796447
  0.37796447 0.37796447 0.         0.37796447 0.37796447 0.
  0.         0.        ]]


# PassiveAgressive Classifier

# Lets Start the Work

In [10]:
import os

In [11]:
import pandas as pd

In [12]:
dataframe = pd.read_csv('news.csv')
dataframe.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [10]:
x = dataframe['text']
y = dataframe['label']

In [11]:
x

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [12]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [15]:
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [16]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

In [17]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

# Finding the Accuracy Score Of the Model

In [18]:
y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.37%


In [19]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[568  47]
 [ 37 615]]


In [20]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [22]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

['REAL']


In [23]:
fake_news_det(
"""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years.""")

['FAKE']


In [24]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [25]:
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [26]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [27]:
fake_news_det1("""
Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years.""")

['FAKE']


In [28]:
fake_news_det1("""
U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.""")

['REAL']


In [29]:
fake_news_det1('''
U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.''')

['REAL']
